---
# Данный файл содержит программу для скачивания через API информации   
# с сайта https://pro.culture.ru   
---     
# Код написан Голомбовским Николаем   
# в рамках прохождения практики в компании Кловери    
---

## 0. Общее описание    

В рамках оценки эффективности различных проектов программы "Пушкинская карта 2.0" (дальнейшее развитие программы "Пушкинская карта"),  
возникла необходимость получить информацию с сайта https://pro.culture.ru о различных организациях культуры на территории России,  
а так же проводимых ими мероприятиям. В этих данных (как по организациям, так и мероприятиям) имеется привязка к программе "Пушкинская карта".  
Планировалось использовать данную информация для разделения новых проектов на те, которые имеют потенциал развития программы "Пушкинская карта", и   
те, которые потеряли актуальность в силу текущего состояния программы "Пушкинская карта".   
Из наиболее перспектиных проектов планировалось формирование программы "Пушкинская карта 2.0".  

Сайт https://pro.culture.ru предоставляет доступ к этой нформации через API.   
При этом существует ограничение на количество получаемых записей из таблиц в рамках одного API-запроса - 100 строк.    
В силу того, что количество организаций Культуры первышает 30 000, а количество проводимых ими мероприятий превышает 2 млн.,  
то использование для анализа прямого доступа к https://pro.culture.ru НЕ является целесообразным.   
Если, например, скачивание всех организаций культуры занимает полтора минуты, то уже скачивание всех мероприятий занимает несколько часов.   
В связи с этим было принято решение вначале перекачать все необходимые данные с https://pro.culture.ru на PostgreSQL, затем выгрузить их   
в формате "csv" и уже после этого закачать эти данные в Power BI для работы аналитика.   

В данном файле проводится демонстрация работы с таблицей "organizations".  
Работа с остальными таблицами проводится по аналогичному алгоритму и НЕ содержит никаких особенностей в коде. Поэтому работа с другими таблицами  
НЕ включена в данный документ.  

Для информации.   
Помимо общей инофрмации с сайта https://pro.culture.ru, имелась так же информация о владельцах Пушкинских карт и использования ими данных карт   
в рамках программы "Пушкинская карта".   
Однако данная информация имеет закрытый статус и не предназначена для общего доступа. Поэтому в данном файле нет никакого упомимнания этих данных.  

## 1. Таблица "organization"    

In [265]:
# Меня учили приводить все загружаемые библиотеки в одном месте в начале программы.
# Выполняю...

import pandas as pd

import json 
import csv
import requests
from pprint import pprint

# Ключ доступа к API Pro.Culture.ru
apiKey = "Здесь должен идти ключ доступа к данным через API, но он исключён по понятным причинам"

import psycopg2

# Доступ к базе данных PostgreSQL и таблице CultureDB
host = '127.0.0.1'
user = 'postgres'
password = 'А это уже пароль к базе данных на моём компьютере - сори, но то же удаляю'
port = '5432'
database = 'CultureDB'

from datetime import datetime, date, timedelta


In [257]:
# Удаляем предыдущую версию таблицы Organization перед её новым заполнением
try:
    connection = psycopg2.connect(
        user = user,
        password = password,
        host = host,
        port = port,
        database = database
    )
    cursor = connection.cursor()
    DropTableQuery = """DROP TABLE Organizations;"""
    cursor.execute(DropTableQuery)
    connection.commit()
    print('Everything is good')
except (Exception, psycopg2.Error) as error:
    print ("Error:", error)
finally:
    if connection:
        cursor.close()
        connection.close()
        print('Connection is closed')

Everything is good
Connection is closed


In [258]:
# Создаём таблицу Organization в базе данных CultureDB
try:
    connection = psycopg2.connect(
        user = user,
        password = password,
        host = host,
        port = port,
        database = database
    )
    cursor = connection.cursor()
    CreateTableQuery = """CREATE TABLE Organizations(
        _id bigint PRIMARY KEY,
        createDate date,
        updateDate date,
        type character varying(20),
        nameShort character varying(300),
        ticketsSold integer,
        status character varying(20),
        category_name character varying(30),
        category_sysName character varying(30),
        inn bigint,
        isPushkinsCard boolean,
        subordination__id smallint,
        subordination_sysName character varying(40),
        subordination_name character varying(40),
        subordination_timezone character varying(30),
        subordination_isYandexExport boolean,
        pushkaRulesAccepted_status boolean,
        organizationTypes character varying(100),
        good_local_id bigint
        );"""
    cursor.execute(CreateTableQuery)
    connection.commit()
    print('Everything is good')
except (Exception, psycopg2.Error) as error:
    print ("Error:", error)
finally:
    if connection:
        cursor.close()
        connection.close()
        print('Connection is closed')

Everything is good
Connection is closed


In [260]:
# делаем ПЕРВЫЙ запрос API в PRO.Культура.ru
params = {"apiKey":apiKey,
          'limit':100, 
          'offset':0}
response = requests.get('https://pro.culture.ru/api/2.5/organizations', params=params) 
metrika_data = response.json()
print(response.status_code)

# Формируем содержимое таблицы Organizations
# Переменная для хранения общего количества организаций
TotalRecordsInTable = metrika_data['total']
for RecNumber in range(TotalRecordsInTable // 100 + 1):
    # Создаём список с нужными полями для вставки в таблицу Organization и сразу порциями по 100 записей делаем вставку в таблицу Organization
    RecordToInsert = []
    # Как оказалось, в базе данных существует НЕСКОЛЬКО организаций с id равным "1". Нужно добавить ТОЛЬКО ОДНУ такую организацию
    SecondTimeId1 = False
    
    # Цикл по всем организациям, скачанным в API-запросе 
    for promRecord in metrika_data['organizations']:
        # Отсечение дублей организации с ID=1
        if promRecord['_id'] == 1:
            if SecondTimeId1: continue
            SecondTimeId1 = True
        
        Rec_createDate = datetime.date(datetime(1970,1,1,0,0,0) + timedelta(seconds=promRecord['createDate'] / 1000))
        Rec_updateDate = datetime.date(datetime(1970,1,1,0,0,0) + timedelta(seconds=promRecord['updateDate'] / 1000))
        
        # Не для каждой организации имеется вся возможная информация. 
        # Поэтому чтобы не допускать ошибки в коде приходится делать проверки на наличие некоторых полей для каждой организации.
        # Следующая переменная содержит названия всех полей для данной организации.
        RecordKeys = promRecord.keys()
        RecordToInsert.append(
            (promRecord['_id'], Rec_createDate, Rec_updateDate, promRecord['type'], 
            #promRecord['nameShort'].replace('\u200b', '').replace('\u200e', '').replace('\u04f1', 'у') if 'nameShort' in RecordKeys else None, 
            promRecord['nameShort'] if 'nameShort' in RecordKeys else None,
            promRecord['ticketsSold'] if 'ticketsSold' in RecordKeys else None, 
            promRecord['status'], 
            promRecord['category']['name'] if 'category' in RecordKeys else None, 
            promRecord['category']['sysName'] if 'category' in RecordKeys else None,
            int(promRecord['inn']) if 'inn' in RecordKeys and promRecord['inn'] != '' else None,
            True if 'isPushkinsCard' in RecordKeys else False,
            promRecord['subordination']['_id'] if promRecord['subordination'] != 0 else None, 
            promRecord['subordination']['sysName'] if promRecord['subordination'] != 0 else None, 
            promRecord['subordination']['name'] if promRecord['subordination'] != 0 else None, 
            promRecord['subordination']['timezone'] if promRecord['subordination'] != 0 else None,
            promRecord['subordination']['isYandexExport'] if promRecord['subordination'] != 0 and 'isYandexExport' in promRecord['subordination'].keys() else None,
            promRecord['pushkaRulesAccepted']['status'] if 'pushkaRulesAccepted' in RecordKeys else None,
            str(promRecord['organizationTypes']) if 'organizationTypes' in RecordKeys else None,
            promRecord['localeIds'][0]
            )
        )

    # Перегнояем данные с сайта Культура в PostgreSQL
    try:
        connection = psycopg2.connect(
            user = user,
            password = password,
            host = host,
            port = port,
            database = database
        )
        cursor = connection.cursor()
        insert_query = """Insert into Organizations (_id, createDate, updateDate, type, nameShort,
            ticketsSold, status, category_name, category_sysName, inn, isPushkinsCard, subordination__id,
            subordination_sysName, subordination_name, subordination_timezone, subordination_isYandexExport,
            pushkaRulesAccepted_status, organizationTypes, good_local_id) 
            Values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
        result = cursor.executemany(insert_query, RecordToInsert)
        connection.commit()

    except (Exception, psycopg2.Error) as error:
        print ("Error:", error)
    finally:
        if connection:
            cursor.close()
            connection.close()
    
    # Следующее условие необходимо для того, чтобы обойти не нужный доступ к сайту PRO.Культура.ru на последней итерации
    if RecNumber == TotalRecordsInTable // 100: break
    
    # закачиваем следующую порцию данных через API из PRO.Культура.РФ и проверяем полученный код на успешность завершения операции
    params = {"apiKey":apiKey,
            'limit':100, 
            'offset':100 * (RecNumber + 1)}
    response = requests.get('https://pro.culture.ru/api/2.5/organizations', params=params) 
    metrika_data = response.json()
    print(response.status_code, params['offset'])
    if response.status_code != 200: break
    
print('Вроде бы всё закачал...')

200
200 100
200 200
200 300
200 400
200 500
200 600
200 700
200 800
200 900
200 1000
200 1100
200 1200
200 1300
200 1400
200 1500
200 1600
200 1700
200 1800
200 1900
200 2000
200 2100
200 2200
200 2300
200 2400
200 2500
200 2600
200 2700
200 2800
200 2900
200 3000
200 3100
200 3200
200 3300
200 3400
200 3500
200 3600
200 3700
200 3800
200 3900
200 4000
200 4100
200 4200
200 4300
200 4400
200 4500
200 4600
200 4700
200 4800
200 4900
200 5000
200 5100
200 5200
200 5300
200 5400
200 5500
200 5600
200 5700
200 5800
200 5900
200 6000
200 6100
200 6200
200 6300
200 6400
200 6500
200 6600
200 6700
200 6800
200 6900
200 7000
200 7100
200 7200
200 7300
200 7400
200 7500
200 7600
200 7700
200 7800
200 7900
200 8000
200 8100
200 8200
200 8300
200 8400
200 8500
200 8600
200 8700
200 8800
200 8900
200 9000
200 9100
200 9200
200 9300
200 9400
200 9500
200 9600
200 9700
200 9800
200 9900
200 10000
200 10100
200 10200
200 10300
200 10400
200 10500
200 10600
200 10700
200 10800
200 10900
200 11000
200 

In [261]:
# Получаем все данные из таблицы Organization из базы данных PostgreSQL
try:
    connection = psycopg2.connect(
        user = user,
        password = password,
        host = host,
        port = port,
        database = database
    )
    cursor = connection.cursor()
    SelectTableQuery = """SELECT * FROM Organizations;"""
    cursor.execute(SelectTableQuery)
    TableResult = cursor.fetchall()
    
    # Как оказалось в переменной "nameShort" имеются так же специальные символы.
    # Данные символы вызывают ошибку при попытке сохранить записи таблицы в csv файл с помощью метода cursor.
    # Поэтому для сохранения записей будет использована библиотека "csv"
    #with open("organizations.csv", "w") as f:
    #    cursor.copy_to(f, 'organizations', sep=',')
    
    connection.commit()
    print('Everything is good')
except (Exception, psycopg2.Error) as error:
    print ("Error:", error)
finally:
    if connection:
        cursor.close()
        connection.close()
        print('Connection is closed')

Everything is good
Connection is closed


In [262]:
# Образец записи по одной из организаций
pprint(TableResult[25551])

(27993,
 datetime.date(2021, 6, 22),
 datetime.date(2023, 10, 28),
 'mincult',
 'МБУДО «Чагодская ДШИ»',
 0,
 'accepted',
 'Образовательные учреждения',
 'obrazovatelnye-uchrezhdeniya',
 352200272855,
 False,
 1992,
 'vologodskiy-rayon',
 'Вологодский район',
 'Europe/Moscow',
 False,
 None,
 None,
 1992)


In [264]:
# Запись csv файлы с полученными данными из таблицы organizations
with open("organizations.csv", mode="w", encoding='utf-8') as w_file:
    file_writer = csv.writer(w_file, delimiter = ",", lineterminator="\r")
    file_writer.writerow(["_id", "createDate", "updateDate", "type", "nameShort",
            "ticketsSold", "status", "category_name", "category_sysName", "inn", "isPushkinsCard", "subordination__id",
            "subordination_sysName", "subordination_name", "subordination_timezone", "subordination_isYandexExport",
            "pushkaRulesAccepted_status", "organizationTypes", "good_local_id"])
    for elem in TableResult:
        file_writer.writerow([str(x) for x in elem])

Всё.   
Файл "organizations.csv" для импорта в Power BI готов...   